In [1]:
(require (calchylus.lambdas [*]))

In [2]:
(with-macros 𝜆 ·)
(assert (= (𝜆 · TRUE) "(𝜆 a b · a)"))

<function <lambda> at 0x000001B1F6B78158>

In [3]:
(with-alpha-conversion-nor-macros L , )
; not lambda expression
(assert (= (, 'x) (, 'x)))
; no body lambda expression
(assert (= (L ,) None))
; constant
(assert (= (L , c) 'c))
; identity, without argument
(assert (= (L x , x) "(L x , x)"))
; identity, with argument
(assert (= (L x , x y) 'y))
; select first, without arguments
(assert (= (L x y , x) "(L x y , x)"))
; select second, without arguments
(assert (= (L x y , y) "(L x y , y)"))
; select first, with arguments
(assert (= (L x y , x 1 0) 1))
; select second, with arguments
(assert (= (L x y , y 1 0) 0))
; function without arguments
(assert (= (L x y z , (x y z)) "(L x y z , (x y z))"))
; nested functions and arguments
(assert (= (L x , (L y , (L z , (z x y) l) k) j) "(l j k)"))
; nested functions, flatten arguments
(assert (= (L x , (L y , (L z , (z x y))) j k l) "(l j k)"))
; flatten functions and arguments
(assert (= (L x y z , (z x y) j k l) "(l j k)"))
; application without arguments
(assert (= (L , ((L x y z , (x y z)))) "(L x y z , (x y z))"))
; application with arguments
(assert (= (L , ((L x y z , (x y z)) a b c)) "(a b c)"))
; free argument
(assert (= (L x , x y z) "(y z)"))
; nested
(assert (= (L , ((L x y z , (x y z) x y z) a b c) 1 2 3) "((x y z) a b c 1 2 3)"))

<function <lambda> at 0x000001B1F6B78510>

In [4]:
(with-alpha-conversion L , )
; variable renaming should give right result
; instead of first substituting x with y, and then both y's with z, resulting (2 2)
; this test should give (y z) because y is bound to x and x only. latter y should not replace it
(assert (= (L x y , (x y) y z) "(y z)"))
; same should apply to deeper values too
(assert (= (L x y , (x y) (y (y z)) z) "((y (y z)) z)"))

<function <lambda> at 0x000001B1F6B9F9D8>

In [5]:
(with-macros L , )
; special macros for main lambda terms / forms
; constant
(assert (= (CONST x) 'x))
; identity, without arguments
(assert (= (IDENT) "(L a , a)"))
; identity, with arguments
(assert (= (IDENT 1) 1))
; identity, nested
(assert (= (IDENT (IDENT 1)) 1))
; boolean macros
(assert (= (, (TRUE) (TRUE 1 0) 
              (FALSE) (FALSE 1 0)) 
           (, "(L a b , a)" 1 
              "(L a b , b)" 0)))

<function <lambda> at 0x000001B1F6EEEA60>

In [6]:
; lists
(assert (= (NIL) "(L a b , b)"))
; pair constructor
(assert (= (PAIR TRUE NIL) "(s (L a b , a) (L a b , b))"))
; selector
(assert (= (PAIR TRUE NIL FIRST) (TRUE)))
; head and tail
(assert (= (, (HEAD (PAIR TRUE NIL)) 
              (TAIL (PAIR TRUE NIL))) 
           (, (TRUE) (NIL))))
; is nil, simple
(assert (= (, (NIL? NIL) (NIL? TRUE)) (, (TRUE) (FALSE))))
; is nil, head and tail
(assert (= (, (NIL? (HEAD (PAIR TRUE NIL))) 
              (NIL? (TAIL (PAIR TRUE NIL)))) 
           (, (FALSE) (TRUE))))
; nested pairs and heads and tails
(assert (=
   (, (HEAD (PAIR TRUE (PAIR TRUE NIL))) 
      (HEAD (TAIL (PAIR TRUE (PAIR TRUE NIL))))
      (TAIL (TAIL (PAIR TRUE (PAIR TRUE NIL)))))
   (, (TRUE) (TRUE) (NIL))))
; nil? for nested pairs
(assert (=
  (, (NIL? (TAIL (TAIL (PAIR TRUE (PAIR TRUE NIL))))) 
     (NIL? (HEAD (TAIL (PAIR TRUE (PAIR TRUE NIL))))))
  (, (TRUE) (FALSE))))
; selectors for pairs
(assert (=  (, (PAIR (PAIR TRUE FALSE FIRST) NIL FIRST)
               (PAIR (PAIR TRUE FALSE SECOND) NIL FIRST)
               (PAIR (PAIR TRUE FALSE FIRST) NIL SECOND)
               (PAIR (PAIR TRUE FALSE SECOND) NIL SECOND)
               (PAIR TRUE (PAIR TRUE NIL FIRST) FIRST)
               (PAIR TRUE (PAIR TRUE NIL SECOND) FIRST)
               (PAIR TRUE (PAIR TRUE NIL FIRST) SECOND)
               (PAIR TRUE (PAIR TRUE NIL SECOND) SECOND))
            (, (TRUE) (FALSE) (NIL) (NIL)
               (TRUE) (TRUE) (TRUE) (NIL))))
; simple condition
(assert (=
    (, (COND TRUE TRUE FALSE) 
       (COND FALSE TRUE FALSE))
    (, (TRUE) (FALSE))))
; number nil? conditions
(assert (=
    (, (COND (NIL? (NUM 0)) TRUE FALSE)
       (COND (NIL? (NUM 1)) TRUE FALSE)
       (COND (NIL? (NUM 10)) TRUE FALSE))
    (, (TRUE) (FALSE) (FALSE))))
; nil tail/head pair condition
(assert (=
    (, (COND (NIL? (TAIL (PAIR (NUM 1) NIL))) TRUE FALSE)
       (COND (NIL? (HEAD (PAIR (NUM 1) NIL))) TRUE FALSE))
    (, (TRUE) (FALSE))))

In [7]:
; church numeral general generator, without arguments
(assert (= (NUM 3) "(L x y , (x (x (x y))))"))
; church numeral, general generator, with arguments
(assert (= (NUM 3 m n) "(m (m (m n)))"))
; church numeral, zero
(assert (= (ZERO) "(L a b , b)"))
; church numeral, zero with arguments
(assert (= (ZERO a b) 'b))
(assert (= (, (ZERO) (ZERO x y) (ONE) (TWO x y) (THREE m n))
           (, "(L a b , b)" "y" "(L x y , (x y))" "(x (x y))" "(m (m (m n)))")))
; if TRUE is given, then the third option is activated!
(assert (= (ZERO? TRUE a b c) 'c))

In [8]:
; letters
(assert (= (LET a 1 b 2 d 3 (a b c d)) "(1 2 c 3)"))
(assert (= (LET a 1 (a a (L a , a 2))) "(1 1 2)"))
(assert (= (LET a 1 b a (a b)) "(1 a)"))
(assert (= (LET a 1 c (a b) (LET b a c)) "(a 1)"))
(assert (= (LET a 1 c (LET b a c) (a b)) "(1 b)"))
(assert (= (LET a 1 b (a b) b) "(a b)"))
(assert (= (LET a 1 (LET b a (a b))) "(1 1)"))
(assert (= (LET* a 1 b a (a b)) "(1 1)"))
(assert (= (LET x) "x"))
(assert (= (LET* x) "x"))
(assert (= (LET) "()"))
(assert (= (LET*) "()"))

In [9]:
; and tests
(assert (=
  (, (AND TRUE TRUE) (AND TRUE FALSE)
     (AND FALSE TRUE) (AND FALSE FALSE))
  (, (TRUE) (FALSE) (FALSE) (FALSE))))
; or tests
(assert (=
  (, (OR TRUE TRUE) (OR TRUE FALSE)
     (OR FALSE TRUE) (OR FALSE FALSE))
  (, (TRUE) (TRUE) (TRUE) (FALSE))))
; not tests
(assert (= (, (NOT TRUE) (NOT FALSE)) (, (FALSE) (TRUE))))
; xor tests
(assert (= 
  (, (XOR TRUE TRUE) (XOR TRUE FALSE)
     (XOR FALSE TRUE) (XOR FALSE FALSE))
  (, (FALSE) (TRUE) (TRUE) (FALSE))))
; logic condition
(assert (= (COND (AND (NOT (XOR FALSE FALSE)) (OR TRUE FALSE)) TRUE FALSE) (TRUE)))
; eval read str
(assert (= (eval (read-str "(TRUE)")) "(L a b , a)"))
; less or equal
(assert (= (, (LEQ? ONE TWO) (LEQ? TWO ONE)  (LEQ? ONE ONE) 
           (, (TRUE) (FALSE) (TRUE)))))
; equal
(assert (= (, (EQ? ONE TWO) (EQ? TWO ONE)  (EQ? ONE ONE) 
           (, (FALSE) (FALSE) (TRUE)))))

In [10]:
; math operations
; next, inc, successive
(assert (= (SUCC ONE) "(x (x y))"))
; add is same as succ
(assert (= (ADD ONE) "(x (x y))"))
; infix notation
(assert (= (ONE ADD ONE ADD ONE) "(x (x (x y)))"))
; previous, dec, predecessive
(assert (= (PRED THREE) "(x (x y))"))
; previous, dec, predecessor, with arguments
(assert (= (PRED THREE a b) "(a (a b))"))
; nested predecessor
(assert (= (PRED (PRED (PRED FOUR))) "(x y)"))
; previous + next is same
(assert (= (SUCC (PRED TWO)) "(x (x y))"))
; previous + next is same for zero
(assert (= (PRED (SUCC ZERO)) 'y))
; but previous + next is one for zero!
(assert (= (SUCC (PRED ZERO)) "(x y)"))
; sum two values
(assert (= (SUM TWO TWO) "(x (x (x (x y))))"))
; substract two x from y
(assert (= (, (SUB ONE TWO) (SUB ONE ONE) (SUB TWO ONE))
           (, "(x y)" 'y 'y)))
(assert (= (, (EXP TWO TWO) (EXP TEN ZERO) (EXP ZERO ZERO))
           (, "(x (x (x (x y))))" "(x y)" "(x y)")))
(assert (= (, (PROD ZERO ONE a b) (PROD ONE ONE a b) (PROD TWO TWO a b))
           (, 'b "(a b)" "(a (a (a (a b))))")))
; self application
(assert (= (SELF (L x , x) 1) 1))
; self application, fixed point
;count down to zero
(assert (= (SELF (L f n , (COND (ZERO? n) ZERO (f f (PRED n)))) THREE) (ZERO)))
; count down to one with lesser or equal comparison
(assert (= (SELF (L f n , (COND (LEQ? n ONE) n (f f (PRED n)))) FOUR) "(x y)"))
; count down to one with equal comparison
(assert (= (SELF (L f n , (COND (EQ? n ONE) n (f f (PRED n)))) FOUR) "(x y)"))

In [11]:
; summation sequence, with plain number
(assert (= (SUMMATION (L x y , (x (x (x y))))) "(x (x (x (x (x (x y))))))"))
; summation sequence, with number macro form
(assert (= (SUMMATION THREE) "(x (x (x (x (x (x y))))))"))
; product sequence, with plain number
(assert (= (FACTORIAL (L x y , (x (x (x y))))) "(x (x (x (x (x (x y))))))"))
; product sequence, with number macro form
(assert (= (FACTORIAL THREE) "(x (x (x (x (x (x y))))))"))
; fibonacci, with plain number
(assert (= (FIBONACCI (L x y , (x (x (x (x (x y))))))) "(x (x (x (x (x (x (x (x (x (x (x (x (x y)))))))))))))"))
; fibonacci, with number macro form
(assert (= (FIBONACCI FIVE) "(x (x (x (x (x (x (x (x (x (x (x (x (x y)))))))))))))"))

In [12]:
; self application recursive loop
(setv result (L x , (x x) (L x , (x x))))
(assert (none? result))

Recursion error occured for lambda expression:  (x , (x x) (L x , (x x)))


In [13]:
(SUMMATION (NUM 8))

'(x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x y))))))))))))))))))))))))))))))))))))'

In [14]:
(FIBONACCI (NUM 9))

'(x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x y)))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))'

In [15]:
; requires alpha conversion and macros, or will cause recursion error!
(with-alpha-conversion-and-macros L , )
(FACTORIAL (NUM 5))

'(x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x y))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))'